<a href="https://colab.research.google.com/github/shamilkv-623/DEA/blob/main/DEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd

# Define the hypothetical dataset
data = {
    'Branch': ['A', 'B', 'C', 'D', 'E'],
    'Doctors': [15, 10, 12, 18, 14],               # Input 1
    'Operating Expenses ($1000)': [120, 100, 110, 150, 130],  # Input 2
    'Patients Treated': [500, 300, 400, 600, 450],  # Output 1
    'Satisfaction Score': [90, 85, 88, 92, 89]      # Output 2
}

# Create a DataFrame
df = pd.DataFrame(data)

# Prepare the input and output arrays
inputs = df[['Doctors', 'Operating Expenses ($1000)']].values
outputs = df[['Patients Treated', 'Satisfaction Score']].values


In [45]:
inputs.shape

(5, 2)

In [47]:
from scipy.optimize import linprog
import numpy as np

def dea_analysis(inputs, outputs):
    num_dmus = inputs.shape[0]
    efficiency_scores = []

    for i in range(num_dmus):
        # Objective function: Minimize theta (efficiency)
        c = [1] + [0] * num_dmus

        # Constraints
        A_eq = [[0] * (1 + num_dmus) for _ in range(1)]  # Normalization constraint
        b_eq = [1]  # Ensure sum of lambdas = 1

        A_ub = []
        b_ub = []

        # Input constraints
        for k in range(inputs.shape[1]):
            constraint = [0] * (1 + num_dmus)
            constraint[0] = inputs[i, k]  # theta * input[i]
            for j in range(num_dmus):
                constraint[j + 1] = -inputs[j, k]  # -lambda[j] * input[j]
            A_ub.append(constraint)
            b_ub.append(0)

        # Output constraints
        for k in range(outputs.shape[1]):
            constraint = [0] * (1 + num_dmus)
            constraint[0] = -outputs[i, k]  # -theta * output[i]
            for j in range(num_dmus):
                constraint[j + 1] = outputs[j, k]  # lambda[j] * output[j]
            A_ub.append(constraint)
            b_ub.append(0)

        # Add normalization constraint
        for j in range(num_dmus):
            A_eq[0][j + 1] = 1

        # Bounds for theta and lambdas
        bounds = [(0, None)] + [(0, None)] * num_dmus

        # Solve the linear program
        result = linprog(c, A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                         A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                         bounds=bounds, method='highs')

        if result.success:
            efficiency_scores.append(result.x[0])  # Efficiency = theta
        else:
            print(f"Solver failed for DMU {i}: {result.message}")
            efficiency_scores.append(None)

    return efficiency_scores


In [48]:
# Calculate efficiency scores
efficiency_scores = dea_analysis(inputs, outputs)

# Add efficiency scores to the DataFrame
df['Efficiency'] = efficiency_scores

# Display the DataFrame with efficiency scores
print(df)


  Branch  Doctors  Operating Expenses ($1000)  Patients Treated  \
0      A       15                         120               500   
1      B       10                         100               300   
2      C       12                         110               400   
3      D       18                         150               600   
4      E       14                         130               450   

   Satisfaction Score  Efficiency  
0                  90    0.991870  
1                  85    1.000000  
2                  88    0.983871  
3                  92    1.000000  
4                  89    1.000000  
